
<style>
u { color: red; }
.red { color: white }
</style>
# <u><span class='red'><br>Subspace Preserving Deep Learning: IonQ Implementation</span></u>


## Introduction

In this notebook, we present our codes to simulate the behavior of Hamming Weight (HW) preserving quantum circuits made of RBS, and CNOTs gates. The simulation are done using python numpy and pytorch library, in a specific subspace by the HW $k$ or a specific encoding which allows one to have better perfomances that using simulation in a IDE that simulate the entire Hilbert space.

We propose to use the encoding algorithm from [arXiv:2309.15547](https://arxiv.org/pdf/2309.15547) to minimize the depth of the quantum circuit. We then perform binary classification using the QCNN for $8 \times 8$ images.

We import here some useful libraries:

In [4]:
import os, sys

sys.path.append(os.path.dirname(os.path.abspath('')))

import numpy as np
import time
from tqdm import tqdm
from scipy.special import binom
import matplotlib.pyplot as plt

Then we load function from our simulation python files:

In [5]:
from IonQ_Simu.toolbox import *
from IonQ_Simu.Encoding_Algorithms import RBS_tensor_encoding

## <mark>Tensor Encoding<mark>

### Definition of the RBS based quantum data loader:

First we define the Quantum Data Loader using our encoding algorithms based on a study of the Quantum Fisher Information Matrix (QFIM). We consider the full connectivity of the IonQ device.

We ask our Algorithm to derive a quantum data loader using $n=16$ qubits with a corresponding HW $k=2$. The connectivity is define as a list of edges called __ListEdges__. Our Encoding algorithm outputs a list of RBS gates. A RBS gate applied between the qubit $i$ and the qubit $j$ is represented as a tuple $(i,j)$.   

Considering tensor encoding, we want to encode images of dimension $I \times I$ with $I=8$. The input are matrices $x \in \mathbb{R}^{I \times I}$. We then need $I^2 - 1$ degrees of freedom.

In [3]:
# Quantum circuit definition:
I = 3
# Connectivity Graph
Qubits = [Vertex("q{}".format(i), i) for i in range(2*I)]
ListEdges = [Edge(Qubits[i],Qubits[j]) for i in range(2*I) for j in range(i+1,2*I)]
Graph_Connectivity = Network(Qubits, ListEdges)

# We apply our encoding algorithm RBS_subspace_encoding_1_heuristic
QDL = RBS_tensor_encoding(I, Graph_Connectivity)

100%|██████████| 8/8 [00:00<00:00, 723.26it/s]

Quantum Data Loader successfully designed


### Fashion MNIST dataset

We import the Fashion MNIST dataset. We preprocess it to have $I \times I$ images. 

In [9]:
import torch
from IonQ_Simu.dataset import load_image_states_fashion_mnist

In [13]:
device = torch.device("mps")

# Data loading parameters:
class_set = [0,1] # We only consider the classes 0 and 1 (binary classification)
train_dataset_number, test_dataset_number = int(1e3), int(1e3)

train_init_states, test_init_states, train_labels, test_labels = load_image_states_fashion_mnist(I, class_set, train_dataset_number, test_dataset_number, device)

# Save the initial states and labels
torch.save(train_init_states, 'train_init_states.pt')
torch.save(test_init_states, 'test_init_states.pt')
torch.save(train_labels, 'train_labels.pt')
torch.save(test_labels, 'test_labels.pt')

Loading training images and labels:


100%|██████████| 1000/1000 [00:00<00:00, 2253.40it/s]


Loading testing images and labels:


100%|██████████| 1000/1000 [00:00<00:00, 2560.53it/s]


In [6]:
from IonQ_Simu.toolbox import RBS_generalized, map_RBS
from src.toolbox import map_RBS_Image_HW2, dictionary_RBS_I2_2D

### Training the data loader:

We simulate our data loader using our torch modules for subspace preserving simulation. We train our data loader for each initial state.

In [6]:
from src.RBS_Circuit import RBS_VQC_state_vector
from IonQ_Simu.Encoding_Algorithms import global_training_quantum_data_loader

In [8]:
Encoding_Load = False
Data_Loading = RBS_VQC_state_vector(2*I, 2, QDL, device)
if Encoding_Load:
    train_encoded_parameters, test_encoded_parameters = global_training_quantum_data_loader(Data_Loading, train_init_states, test_init_states, device)
    torch.save(train_encoded_parameters, 'train_encoded_parameters.pt')
    torch.save(test_encoded_parameters, 'test_encoded_parameters.pt')

We can test the quality of the data loading:

In [9]:
# Initial state (corresponding to initial bit flips):
initial_state = torch.zeros(int(binom(2*I,2)), device=device)
initial_state[0] = 1

# Change of basis:
I = int(np.sqrt(train_init_states.shape[1]))
dict_I2, map_RBS_HW2 = dictionary_RBS_I2_2D(I), map_RBS(2*I,2)

sample = torch.from_numpy(map_RBS_Image_HW2(I, dict_I2, map_RBS_HW2, test_init_states[-1])).type(torch.float32).to(device)
print(torch.nn.MSELoss()(Data_Loading(initial_state), sample))

tensor(0.1699, device='mps:0', grad_fn=<MseLossBackward0>)


## <mark>Quantum Convolutional Neural Network architecture<mark>

### Defining the architecture and training it

In this Section, we define the architecture and we train it. First, let us load some useful python libraries and define the hyperparameters of the model:

In [10]:
from src.list_gates import drip_circuit, full_pyramid_circuit
from src.load_dataset import load_fashion_mnist
from src.training import train_globally_2D
from IonQ_Simu.QCNN_2D import QCNN

from torch.optim.lr_scheduler import ExponentialLR


In [11]:
# Below are the hyperparameters of this network, you can change them to test
I = 3  # dimension of image we use. If you use 2 times conv and pool layers, please make it a multiple of 4
O = I // 2  # dimension after pooling, usually you don't need to change this
K = 2  # size of kernel in the convolution layer, please make it divisible by O=I/2
batch_size = 10  # batch number
reduced_qubit = 3 # ATTENTION: let binom(reduced_qubit,k) >= len(class_set)!
is_shuffle = False  # shuffle for this dataset
learning_rate = 1e-1  # step size for each learning steps
train_epochs = 10  # number of epoch we train
test_interval = 1  # when the training epoch reaches an integer multiple of the test_interval, print the testing result
criterion = torch.nn.CrossEntropyLoss()  # loss function
device = torch.device("mps")  # also torch.device("cpu"), or torch.device("cuda") for gpus
output_scale = 10

# Dense part:
dense_full_gates = drip_circuit(I)
dense_reduce_gates = full_pyramid_circuit(reduced_qubit)

We use our pytorch modules to simulate a Quantum Convolutional Neural Network architecture. 

In [14]:
network = QCNN(I, O, dense_full_gates, dense_reduce_gates, reduced_qubit, device)
optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
scheduler = ExponentialLR(optimizer, gamma=0.9)

# Loading data
train_dataloader, test_dataloader = load_fashion_mnist(class_set, train_dataset_number, test_dataset_number, batch_size)
# train_dataloader, test_dataloader = load_mnist(class_set, train_dataset_number, test_dataset_number, batch_size)

# training part
network_state = train_globally_2D(batch_size, I, network, train_dataloader, test_dataloader, optimizer, scheduler, criterion, output_scale, train_epochs, test_interval, device)

: 

## <mark>PennyLane Simulation<mark>

In this Section, we define the PennyLane simulations of our quantum encoding and machine learning algorithms.

In [ ]:
import pennylane as qml

from Verification_correction.PennyLane_toolbox import RBS, Conv_RBS_2D, Pool_2D, dense_RBS
from src.toolbox import QCNN_RBS_based_VQC

We define the global circuit (encoding + qcnn) using the pennylane library. The circuit should be the same as described in the previous sections. 

In [ ]:
dev = qml.device("default.qubit", wires=2*I)

def Global_QCNN_Circuit(I, K, angles_QDL, angles_conv, angles_dense):
    # Initial state preparation
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    # Quantum data loader
    for i,(a,b) in enumerate(QDL):
        RBS(a, b, angles_QDL[i])
    # Convolutional layer
    Conv_RBS_2D(angles_conv, I, K) 
    # Pooling layer
    Pool_2D([i for i in range(2*I)])
    # Dense layer
    for index,(i,j) in enumerate(dense_full_gates):
        RBS(i*2 +1, j*2 +1, angles_dense[index])
    for index, (i,j) in enumerate(dense_reduce_gates):
        RBS(i*2 +1, j*2 +1, angles_dense[len(dense_full_gates)+index])